# Quickstart for Carsus

In this guide you will learn how to create an atomic data file suitable for [TARDIS](https://github.com/tardis-sn/tardis).

## Atomic Weights and Ionization Energies

Atomic weights and ionization energies are taken from the National Institute of Standards and Technology (NIST).

<div class="alert alert-info">

**See also:**

Get familiar with [Notation in Carsus](development/notation.rst).
    
</div>

In [ ]:
from carsus.io.nist import NISTWeightsComp, NISTIonizationEnergies

atomic_weights = NISTWeightsComp()
ionization_energies = NISTIonizationEnergies('H-Zn')

## Levels, Lines and Collisions

Carsus supports three sources of energy levels and spectral lines: GFALL, CHIANTI and CMFGEN.

### GFALL

The Robert Kurucz's Atomic Linelist (GFALL) is the main source of levels and lines.

In [ ]:
from carsus.io.kurucz import GFALLReader

gfall_reader = GFALLReader('H-Zn')

<div class="alert alert-warning">

**Warning:**
    
Creating a `GFALLReader` instance is **required**. Other sources of levels and lines are **optional**.

</div>

### CHIANTI

The Chianti Atomic Database is also a source of collision strengths.

In [ ]:
from carsus.io.chianti_ import ChiantiReader

chianti_reader = ChiantiReader('H-He', collisions=True, priority=20)

By default `priority` parameter is set to `10`. Increase this value if you want to keep CHIANTI levels and lines over GFALL.

### CMFGEN

Ingestion of energy levels and spectral lines from the CMFGEN Atomic Data is not so straightforward.

You'll need to parse the files manually, construct a dictionary and create a `CMFGENReader` object.

In [ ]:
from carsus.io.cmfgen import CMFGENReader, CMFGENEnergyLevelsParser, CMFGENOscillatorStrengthsParser

# Parse levels
si_0_lvl = CMFGENEnergyLevelsParser('/tmp/atomic/SIL/I/23nov11/SiI_OSC')
si_1_lvl = CMFGENEnergyLevelsParser('/tmp/atomic/SIL/II/16sep15/si2_osc_kurucz')

# Parse lines
si_0_osc = CMFGENOscillatorStrengthsParser('/tmp/atomic/SIL/I/23nov11/SiI_OSC')
si_1_osc = CMFGENOscillatorStrengthsParser('/tmp/atomic/SIL/II/16sep15/si2_osc_kurucz')

# Create dict
cmfgen_data = {'Si 0': {'levels': si_0_lvl, 'lines': si_0_osc},
               'Si 1': {'levels': si_1_lvl, 'lines': si_1_osc},}

cmfgen_reader = CMFGENReader(cmfgen_data, priority=20)

## Zeta Data

The Knox S. Long's ground state recombinations fractions ($\zeta$).

In [ ]:
from carsus.io.zeta import KnoxLongZeta

zeta_data = KnoxLongZeta()

## Dump to HDF5

Finally, create a `TARDISAtomData` object and dump the data with the `to_hdf` method.

In [ ]:
from carsus.io.output import TARDISAtomData

atom_data = TARDISAtomData(atomic_weights,
                           ionization_energies,
                           gfall_reader,
                           zeta_data,
                           chianti_reader,
                           cmfgen_reader)

In [ ]:
atom_data.to_hdf('kurucz_cd23_chianti_H-He_cmfgen_Si_I-II.h5')

You are done! Now you can use your file to run TARDIS simulations.

## Metadata

Carsus stores metadata inside the HDF5 files to ensure reproducibility. This metadata includes a checksum per `DataFrame`, the current version of every dataset and relevant software versions. 

In [ ]:
import pandas as pd

In [ ]:
pd.read_hdf('kurucz_cd23_chianti_H-He_cmfgen_Si_I-II.h5', key='meta')

In [ ]:
# nbsphinx hidden cell
!rm kurucz_cd23_chianti_H-He.h5